In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

import nltk
import nltk.sentiment
import re

from requests import get
from bs4 import BeautifulSoup
import os

import acquire
import prepare

from sklearn.feature_extraction.text import TfidfVectorizer, \
                                            CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, accuracy_score

In [2]:
#acquire.acquire_repo_list('petroleum')

In [3]:
#acquire.acquire_repo_list('solar energy')

In [4]:
#acquire.acquire_repo_list('natural gas')

In [5]:
#df = acquire.scrape_github_data()

In [6]:
df = pd.read_csv('new_repos_dict.csv')

In [7]:
df.loc[:200, 'category'] = 'gasoline'

In [8]:
df.loc[200:400, 'category'] = 'wind'

In [9]:
df.loc[400:, 'category'] = 'solar'

In [10]:
df = prepare.prep_repo_data(df)

In [11]:
df.dropna(inplace=True)

In [12]:
df.reset_index(drop=True, inplace=True)

In [13]:
df

,language,category,repo,readme_contents,clean_tokes,clean_stemmed,clean_lemmatized
0,C,gasoline,N-BodyShop/gasoline,```\n \t ▄████ ▄▄▄ ██████ ▒█████ █...,"[&#9;, &#9;, &#9;, &#9;, &#9;, &#9;, &#9;, &#9...",&#9; &#9; &#9; &#9; &#9; &#9; &#9; &#9; &#9; g...,&#9; &#9; &#9; &#9; &#9; &#9; &#9; &#9; &#9; g...
1,OCaml,gasoline,michipili/gasoline,# Gasoline\n\nThe Gasoline project aims at imp...,"[gasoline, the, gasoline, project, aims, at, i...",gasolin gasolin project aim implement unixish ...,gasoline gasoline project aim implementing uni...
2,TypeScript,gasoline,rvikmanis/gasoline,# Gasoline\n\nConvenient state container for R...,"[gasoline, convenient, state, container, for, ...",gasolin conveni state contain react instal npm...,gasoline convenient state container react inst...
3,Python,gasoline,iggisv9t/benzin_gif,# benzin_gif\nCreate animated gifs that looks ...,"[benzin_gif, create, animated, gifs, that, loo...",benzin_gif creat anim gif look like gasolin pu...,benzin_gif create animated gifs look like gaso...
4,Go,gasoline,daneharrigan/gasoline,# gasoline\n\n![Gasoline Dashboard](http://cl....,"[gasoline, gasoline, dashboardhttpcllyimage3l1...",gasolin gasolin dashboardhttpcllyimage3l190u3q...,gasoline gasoline dashboardhttpcllyimage3l190u...
...,...,...,...,...,...,...,...
405,JavaScript,solar,sukanyavenkataraman/enerhack,# Enerhack\n\nCode for communicating between t...,"[enerhack, code, for, communicating, between, ...",enerhack code commun energyan dahsboard,enerhack code communicating energyan dahsboard
406,Jupyter Notebook,solar,souvikb07/solar-energy-cost-calculator-models,# solar-energy-cost-calculator-models\n\n### W...,"[solarenergycostcalculatormodels, work, in, pr...",solarenergycostcalculatormodel work progress,solarenergycostcalculatormodels work progress
407,CSS,solar,suwalka/ushasolarpower,# usp\nClient site based on solar energy produ...,"[usp, client, site, based, on, solar, energy, ...",usp client site base solar energi product,usp client site based solar energy product
408,HTML,solar,kerrongordon/gsetri,# Generator-jekyllrb \n\n[![Build Status](http...,"[generatorjekyllrb, build, statushttpstravisci...",generatorjekyllrb build statushttpstravisciorg...,generatorjekyllrb build statushttpstravisciorg...


In [14]:
df.clean_lemmatized.apply(len).nlargest(15)

135    84881
269    84881
114    81494
388    81123
17     64273
234    31806
270    16389
130    11005
297    10445
14      9942
330     7553
320     7272
192     6802
264     6776
292     6537
Name: clean_lemmatized, dtype: int64

In [15]:
df = df[df.index!=135]

In [16]:
df = df[df.index!=269]

In [17]:
df['word_count'] = df.clean_lemmatized.str.split().apply(len)

In [18]:
df.language.value_counts()

Jupyter Notebook     67
JavaScript           57
Python               52
PowerShell           36
HTML                 30
Java                 22
C++                  19
C                    18
R                    14
PHP                  14
CSS                  10
C#                    9
MATLAB                8
TypeScript            6
Matlab                5
Ruby                  5
Arduino               3
Swift                 3
Processing            2
Eagle                 2
Batchfile             2
Go                    2
Objective-C           2
Dart                  2
TeX                   2
OCaml                 1
Fortran               1
Scala                 1
Vue                   1
Visual Basic .NET     1
Perl                  1
Makefile              1
CMake                 1
Modelica              1
Lua                   1
TSQL                  1
PostScript            1
M                     1
Cuda                  1
ActionScript          1
Visual Basic          1
Name: language, 

In [19]:
df = df.replace('Matlab', 'MATLAB')

In [20]:
df.language.value_counts()

Jupyter Notebook     67
JavaScript           57
Python               52
PowerShell           36
HTML                 30
Java                 22
C++                  19
C                    18
R                    14
PHP                  14
MATLAB               13
CSS                  10
C#                    9
TypeScript            6
Ruby                  5
Swift                 3
Arduino               3
Batchfile             2
Eagle                 2
Go                    2
Objective-C           2
TeX                   2
Dart                  2
Processing            2
Fortran               1
Scala                 1
Vue                   1
Visual Basic .NET     1
OCaml                 1
Perl                  1
CMake                 1
Makefile              1
Lua                   1
TSQL                  1
PostScript            1
Visual Basic          1
M                     1
Cuda                  1
ActionScript          1
Modelica              1
Name: language, dtype: int64

In [ ]:
plt.figure(figsize=(16, 9))
pd.Series(df.clean_tokes).value_counts().head(12).plot.barh(width=.9, ec='black', title='12 most common words')
plt.xlabel('Number of occurances')

In [ ]:
#ADDITIONAL_STOPWORDS = ['r', 'u', '2', 'ltgt']

def clean(text: str) -> list:
    'A simple function to cleanup text data'
    wnl = nltk.stem.WordNetLemmatizer()
    stopwords = nltk.corpus.stopwords.words('english')
    text = (text.encode('ascii', 'ignore')
             .decode('utf-8', 'ignore')
             .lower())
    words = re.sub(r'[^\w\s]', '', text).split() # tokenization
    return [wnl.lemmatize(word) for word in words if word not in stopwords]

all_words = clean(' '.join(df.readme_contents))

plt.figure(figsize=(16, 9))
pd.Series(all_words).value_counts().head(12).plot.barh(width=.9, ec='black', title='12 most common words')
plt.xlabel('Number of occurances')

In [ ]:
df.clean_stemmed.str.len()

In [ ]:
df.info()

In [ ]:
df.describe()

In [23]:
top_lang = list(df.language.value_counts().head(12).index)
top_lang

['Jupyter Notebook',
 'JavaScript',
 'Python',
 'PowerShell',
 'HTML',
 'Java',
 'C++',
 'C',
 'R',
 'PHP',
 'MATLAB',
 'CSS']

In [ ]:
plt.figure(figsize=(18,9))
sns.boxplot(data=df, y=df.word_count, x='language')
plt.title("README Word Length by programing language")
plt.xlabel('Programming Language')
plt.ylabel('README Length')
plt.show()

In [ ]:
df[df.language == 'Jupyter Notebook']

In [ ]:
low = df.language.value_counts() < 5

In [ ]:
[low.index[i] for i, x in enumerate(low) if x]

In [26]:
words_250 = df[df.word_count > 250]

In [37]:
words_250

,language,category,repo,readme_contents,clean_tokes,clean_stemmed,clean_lemmatized,word_count
1,OCaml,gasoline,michipili/gasoline,# Gasoline\n\nThe Gasoline project aims at imp...,"[gasoline, the, gasoline, project, aims, at, i...",gasolin gasolin project aim implement unixish ...,gasoline gasoline project aim implementing uni...,345
7,Jupyter Notebook,gasoline,madsenmj/ml-gas-price,# Gas Price Prediction Model\n\nThis project u...,"[gas, price, prediction, model, this, project,...",ga price predict model thi project use public ...,gas price prediction model project us public d...,564
10,Ruby,gasoline,Jenyaya/Gasoline,== Welcome to Rails\n\nRails is a web-applicat...,"[welcome, to, rails, rails, is, a, webapplicat...",welcom rail rail webappl framework includ ever...,welcome rail rail webapplication framework inc...,753
12,Ruby,gasoline,charles0906/gasolineapp,== Welcome to Rails\n\nRails is a web-applicat...,"[welcome, to, rails, rails, is, a, webapplicat...",welcom rail rail webappl framework includ ever...,welcome rail rail webapplication framework inc...,748
14,JavaScript,gasoline,abuenosvinos/gasolineras-www,\n# Proyecto Gasolineras\n\n[![Author][Author]...,"[proyecto, gasolineras, authorauthorhttpwwwant...",proyecto gasolinera authorauthorhttpwwwantonio...,proyecto gasolineras authorauthorhttpwwwantoni...,1431
17,JavaScript,gasoline,JoelLappalainen/split-gasoline-costs,This project was bootstrapped with [Create Rea...,"[this, project, was, bootstrapped, with, creat...",thi project wa bootstrap creat react apphttpsg...,project wa bootstrapped create react apphttpsg...,7172
38,R,gasoline,akrm3008/Gasoline,# Introduction\nFew days before landfall of Hu...,"[introduction, few, days, before, landfall, of...",introduct day befor landfal hurrican irma flor...,introduction day landfall hurricane irma flori...,689
41,PHP,gasoline,CamilloBovio/Gasoline-Rec.,\nCONTENTS OF THIS FILE\n---------------------...,"[contents, of, this, file, about, drupal, conf...",content thi file drupal configur featur appear...,content file drupal configuration feature appe...,272
81,Objective-C,gasoline,omedbb/write-goose-gasoline,# 2048\n\nThis is a derivative and the iOS ver...,"[2048, this, is, a, derivative, and, the, ios,...",2048 thi deriv io version game 2048 veri unlik...,2048 derivative io version game 2048 unlikely ...,563
97,Objective-C,gasoline,omedbb/coal-greatest-gasoline,# 2048\n\nThis is a derivative and the iOS ver...,"[2048, this, is, a, derivative, and, the, ios,...",2048 thi deriv io version game 2048 veri unlik...,2048 derivative io version game 2048 unlikely ...,563


In [38]:
words_250.language.value_counts()

Python              15
Jupyter Notebook    11
JavaScript           6
PowerShell           3
Java                 3
R                    3
CSS                  2
MATLAB               2
C++                  2
Ruby                 2
HTML                 2
Objective-C          2
TSQL                 1
PostScript           1
C                    1
OCaml                1
Visual Basic         1
C#                   1
PHP                  1
Name: language, dtype: int64

In [27]:
documents = words_250[['repo','language', 'clean_lemmatized']]

In [28]:
documents

,repo,language,clean_lemmatized
1,michipili/gasoline,OCaml,gasoline gasoline project aim implementing uni...
7,madsenmj/ml-gas-price,Jupyter Notebook,gas price prediction model project us public d...
10,Jenyaya/Gasoline,Ruby,welcome rail rail webapplication framework inc...
12,charles0906/gasolineapp,Ruby,welcome rail rail webapplication framework inc...
14,abuenosvinos/gasolineras-www,JavaScript,proyecto gasolineras authorauthorhttpwwwantoni...
17,JoelLappalainen/split-gasoline-costs,JavaScript,project wa bootstrapped create react apphttpsg...
38,akrm3008/Gasoline,R,introduction day landfall hurricane irma flori...
41,CamilloBovio/Gasoline-Rec.,PHP,content file drupal configuration feature appe...
81,omedbb/write-goose-gasoline,Objective-C,2048 derivative io version game 2048 unlikely ...
97,omedbb/coal-greatest-gasoline,Objective-C,2048 derivative io version game 2048 unlikely ...


In [55]:
tfidf = TfidfVectorizer()
X = tfidf.fit_transform(documents.clean_lemmatized)
y = documents.language

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=.8)

train = pd.DataFrame(dict(actual=y_train))
test = pd.DataFrame(dict(actual=y_test))

model = LogisticRegression().fit(X_train, y_train)
#model = DecisionTreeClassifier(max_depth=6, random_state=42).fit(X_train, y_train)
#model = RandomForestClassifier(max_depth=6, min_samples_leaf=2, random_state=42).fit(X_train, y_train)
#model = KNeighborsClassifier().fit(X_train, y_train)

train['predicted'] = model.predict(X_train)
test['predicted'] = model.predict(X_test)

print(round(model.score(X_train, y_train), 4))
print(round(model.score(X_test, y_test), 4))

0.5833
0.3333


In [56]:
cv = CountVectorizer()
X = cv.fit_transform(documents.clean_lemmatized)
y = documents.language

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=.8)

train = pd.DataFrame(dict(actual=y_train))
test = pd.DataFrame(dict(actual=y_test))

model = LogisticRegression().fit(X_train, y_train)
#model = DecisionTreeClassifier(max_depth=6, random_state=42).fit(X_train, y_train)
#model = RandomForestClassifier(max_depth=6, min_samples_leaf=2, random_state=42).fit(X_train, y_train)
#model = KNeighborsClassifier().fit(X_train, y_train)


train['predicted'] = model.predict(X_train)
test['predicted'] = model.predict(X_test)

print(round(model.score(X_train, y_train), 4))
print(round(model.score(X_test, y_test), 4))

1.0
0.4167


/usr/local/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [58]:
tfidf = TfidfVectorizer(ngram_range=(2, 2))
X = tfidf.fit_transform(documents.clean_lemmatized)
y = documents.language

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=.8)

train = pd.DataFrame(dict(actual=y_train))
test = pd.DataFrame(dict(actual=y_test))

model = LogisticRegression().fit(X_train, y_train)
#model = DecisionTreeClassifier(max_depth=6, random_state=42).fit(X_train, y_train)
#model = RandomForestClassifier(max_depth=6, min_samples_leaf=2, random_state=42).fit(X_train, y_train)
#model = KNeighborsClassifier().fit(X_train, y_train)

train['predicted'] = model.predict(X_train)
test['predicted'] = model.predict(X_test)

print(round(model.score(X_train, y_train), 4))
print(round(model.score(X_test, y_test), 4))

0.5833
0.1667


In [63]:
cv = CountVectorizer(ngram_range=(2, 2))
X = cv.fit_transform(documents.clean_lemmatized)
y = documents.language

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=.8)

train = pd.DataFrame(dict(actual=y_train))
test = pd.DataFrame(dict(actual=y_test))

#model = LogisticRegression().fit(X_train, y_train)
model = DecisionTreeClassifier(max_depth=6, random_state=42).fit(X_train, y_train)
#model = RandomForestClassifier(max_depth=6, min_samples_leaf=2, random_state=42).fit(X_train, y_train)
#model = KNeighborsClassifier().fit(X_train, y_train)


train['predicted'] = model.predict(X_train)
test['predicted'] = model.predict(X_test)

print(round(model.score(X_train, y_train), 15))
print(round(model.score(X_test, y_test), 15))

0.5625
0.25


In [33]:
test.shape

(12, 2)

In [34]:
train.shape

(48, 2)

In [ ]:
test